<a href="https://colab.research.google.com/github/Harshithackz/cryptocurrency-data-analysis/blob/main/WebScraping_Crypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Import Libraries
import requests
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [8]:
# The List of 20 type of cryptocurrency
coin_list = ['BTC', 'ETH', 'XRP', 'ADA', 'USDT', 'DOGE', 'XLM', 'DOT', 'UNI', 'LINK', 'USDC', 'BCH', 'LTC', 'GRT', 'ETC', 'FIL', 'AAVE', 'ALGO', 'EOS']

In [9]:
#defining the dataframe
main_df = pd.DataFrame()

In [10]:
for coin in coin_list:
    coin_df = pd.DataFrame()
    df = pd.DataFrame(index=[0])

    # Defining the Start Date and End Date(single day taken here)
    datetime_end = datetime(2024, 8, 2, 0, 0)
    datetime_check = datetime(2024, 8, 1, 0, 0)

    while len(df) > 0:
        if datetime_end == datetime_check:
            break

        datetime_start = datetime_end - relativedelta(hours = 12)

        #Api for the scrapping from coindesk
        url = 'https://production.api.coindesk.com/v2/price/values/'+ coin +'?start_date='+datetime_start.strftime("%Y-%m-%dT%H:%M") + '&end_date=' + datetime_end.strftime("%Y-%m-%dT%H:%M") + '&ohlc=true'

        #we are using the request to fetch the data from the api in the json format and then storing it into the dataframe.
        temp_data = requests.get(url).json()
        df = pd.DataFrame(temp_data['data']['entries'])
        df.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close']

        # To handle the Missing Data
        insert_ids_list = [np.nan]

        '''In numerous occasions, I observed that CoinDesk did not collect every minute of data.
        According to observation, in a normal circumstance, the timestamp would have a discrepancy
        value of 60000 for 1 minute. As a result, I can quickly tell that the missing period is the
        time gap if the row difference for the timestamp is larger than 60000. To deal with it, a
        hot-deck imputation procedure is used. To put it another way, the nearest minute data will
        be substituted for the missing one.
        '''

        while len(insert_ids_list) > 0:
            timestamp_checking = np.array(df['Timestamp'][1:]) - np.array(df['Timestamp'][:-1])
            insert_ids_list = np.where(timestamp_checking!= 60000)[0]
            if len(insert_ids_list) > 0:
                print(str(len(insert_ids_list)) + ' mismatched.')
                insert_ids = insert_ids_list[0]
                temp_df = df.iloc[insert_ids.repeat(int(timestamp_checking[insert_ids]/60000)-1)].reset_index(drop=True)
                temp_df['Timestamp'] = [temp_df['Timestamp'][0] + i*60000 for i in range(1, len(temp_df)+1)]
                df = pd.concat([df.loc[:insert_ids], temp_df, df.loc[insert_ids+1:]]).reset_index(drop=True)
                insert_ids_list = insert_ids_list[1:]


        #adding datetime and symbol to dataframe
        df = df.drop(['Timestamp'], axis=1)
        df['Datetime'] = [datetime_end - relativedelta(minutes=len(df)-i) for i in range(0, len(df))]
        coin_df = pd.concat([df, coin_df])
        datetime_end = datetime_start

    coin_df['Symbol'] = coin
    main_df = pd.concat([main_df, coin_df])


12 mismatched.
11 mismatched.
10 mismatched.
9 mismatched.
8 mismatched.
7 mismatched.
6 mismatched.
5 mismatched.
4 mismatched.
3 mismatched.
2 mismatched.
1 mismatched.
12 mismatched.
11 mismatched.
10 mismatched.
9 mismatched.
8 mismatched.
7 mismatched.
6 mismatched.
5 mismatched.
4 mismatched.
3 mismatched.
2 mismatched.
1 mismatched.
12 mismatched.
11 mismatched.
10 mismatched.
9 mismatched.
8 mismatched.
7 mismatched.
6 mismatched.
5 mismatched.
4 mismatched.
3 mismatched.
2 mismatched.
1 mismatched.
12 mismatched.
11 mismatched.
10 mismatched.
9 mismatched.
8 mismatched.
7 mismatched.
6 mismatched.
5 mismatched.
4 mismatched.
3 mismatched.
2 mismatched.
1 mismatched.
12 mismatched.
11 mismatched.
10 mismatched.
9 mismatched.
8 mismatched.
7 mismatched.
6 mismatched.
5 mismatched.
4 mismatched.
3 mismatched.
2 mismatched.
1 mismatched.
12 mismatched.
11 mismatched.
10 mismatched.
9 mismatched.
8 mismatched.
7 mismatched.
6 mismatched.
5 mismatched.
4 mismatched.
3 mismatched.
2 

In [11]:
main_df = main_df[['Datetime', 'Symbol', 'Open', 'High', 'Low', 'Close']].reset_index(drop=True)
main_df

,Datetime,Symbol,Open,High,Low,Close
0,2024-07-31 23:59:00,BTC,64694.314641,64887.036065,64530.836412,64758.127805
1,2024-08-01 00:00:00,BTC,64694.314641,64887.036065,64530.836412,64758.127805
2,2024-08-01 00:01:00,BTC,64694.314641,64887.036065,64530.836412,64758.127805
3,2024-08-01 00:02:00,BTC,64694.314641,64887.036065,64530.836412,64758.127805
4,2024-08-01 00:03:00,BTC,64694.314641,64887.036065,64530.836412,64758.127805
...,...,...,...,...,...,...
27393,2024-08-01 23:55:00,EOS,0.544179,0.553117,0.544105,0.551657
27394,2024-08-01 23:56:00,EOS,0.544179,0.553117,0.544105,0.551657
27395,2024-08-01 23:57:00,EOS,0.544179,0.553117,0.544105,0.551657
27396,2024-08-01 23:58:00,EOS,0.544179,0.553117,0.544105,0.551657


In [12]:
main_df.to_csv('main_df.csv', index=False)